In [1]:
from datetime import datetime, time, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import xarray as xr
from ocf_blosc2 import Blosc2
from torch.utils.data import DataLoader, IterableDataset
from torchinfo import summary
import json
import glob
plt.rcParams["figure.figsize"] = (20, 12)
# %load_ext autoreload
# %autoreload 2


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
from dataset import HDF5Dataset
dataset = HDF5Dataset(['/data/processed_data/processed_train_1.hdf5'], "/data/sat_np/", "/data/weather_np/", True, True, True, True)
data_loader = DataLoader(dataset, batch_size=16, pin_memory=True, num_workers=6, shuffle=True)
print(f"train dataset len: {len(dataset)}")

Opening file /data/processed_data/processed_train_1.hdf5.
Warming up the dataloader!


100%|██████████| 3447/3447 [00:00<00:00, 144394.28it/s]

train dataset len: 3447


In [4]:
EPOCHS = 15
START_EPOCH = 0
LR = 1e-3
from submission.model import OurResnet2
model = OurResnet2(image_size=128, device=device).to(device)
criterion = nn.L1Loss()
optimiser = optim.AdamW(model.parameters(), lr=LR, weight_decay=0.02)
lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimiser, T_max=10, eta_min=7e-5)
summary(model, input_size=[(1, 12), (1, 11, 12, 128, 128), (1, 10, 6, 128, 128), (1, 3)])
# x = torch.randn((1, 12)).to(device)
# y = torch.randn((1, 1, 12, 128, 128)).to(device)
# z = torch.randn((1, 10, 6, 128, 128)).to(device)
# a = torch.randn((1, 3)).to(device)
# model(x, y, z, a)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
def hasNan(tensor):
    return torch.isnan(tensor).any()

In [ ]:
MODEL_KEY="ExtraEmbedding_TemporalResnet2+1Combo-PVResFCNet2"
print(f"Training model key {MODEL_KEY}")
from tqdm import tqdm
for epoch in range(EPOCHS):
    model.train()

    running_loss = 0.0
    i = 0
    count = 0
    for (pv_features, hrv_features, nwp, extra, pv_targets) in (pbar := tqdm(data_loader, total=len(data_loader), ascii=True)):
        optimiser.zero_grad()
        with torch.autocast(device_type="cuda"):
            real_extra = extra[:, 2:]
            if hasNan(pv_features) or hasNan(hrv_features) or hasNan(nwp) or hasNan(extra) or hasNan(pv_targets):
                print(f"Found nan {i}")
                continue
            predictions = model(
                pv_features.to(device,dtype=torch.float),
                hrv_features.to(device,dtype=torch.float),
                nwp.to(device,dtype=torch.float),
                real_extra.to(device,dtype=torch.float),
            )
            # print(pv_features.shape, hrv_features.shape, nwp.shape, real_extra.shape)
            loss = criterion(predictions, pv_targets.to(device, dtype=torch.float))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimiser.step()

        size = int(pv_targets.size(0))
        running_loss += float(loss) * size
        count += size

        if i % 10 == 9:
            writer.add_scalar(f"Loss/train_batch_level", (running_loss / count), epoch * len(data_loader) + i)
            pbar.set_description(f"Epoch {START_EPOCH + epoch + 1}, {i + 1}: {running_loss / count}")
        if i % 100 == 99:
            print(f"Epoch {START_EPOCH + epoch + 1}, {i + 1}: {running_loss / count}")
            writer.add_scalar(f"Loss/train_ep_level", (running_loss / count), START_EPOCH + epoch + 1)
        if i % 3000 == 2999:
            torch.save(model.state_dict(), f"./cpts/{MODEL_KEY}-ep{START_EPOCH + epoch + 1}.pt")
        i += 1
    lr_scheduler.step() 
    current_lr = lr_scheduler.get_last_lr()[0]
    print(f"Epoch {START_EPOCH + epoch + 1}: {running_loss / count} (LR: {current_lr})")
    writer.add_scalar(f"LR", current_lr, START_EPOCH + epoch + 1)
    torch.save(model.state_dict(), f"./cpts/{MODEL_KEY}-ep{START_EPOCH + epoch + 1}.pt")
    print("Saved model!")

Training model key ExtraEmbedding_TemporalResnet2+1Combo-PVResFCNet2


Epoch 1, 100: 0.09786068400368095:  23%|##3       | 100/431 [01:56<05:25,  1.02it/s]

Epoch 1, 100: 0.09786068400368095


Epoch 1, 200: 0.08536064063198864:  46%|####6     | 200/431 [03:37<03:57,  1.03s/it]

Epoch 1, 200: 0.08536064063198864


Epoch 1, 300: 0.08370284426336487:  70%|######9   | 300/431 [05:22<02:32,  1.16s/it]

Epoch 1, 300: 0.08370284426336487


Epoch 1, 340: 0.08166697598884211:  81%|########  | 349/431 [06:12<01:27,  1.07s/it]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 5.
Original Traceback (most recent call last):
  File "/home/dsingh/miniconda3/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/dsingh/miniconda3/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dsingh/miniconda3/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/home/dsingh/source/devksingh4/climatehack-2023/vit/dataset.py", line 89, in __getitem__
    arr = np.load(total_name, allow_pickle=True)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dsingh/miniconda3/lib/pyth

In [ ]:
for i in range(100):
    pv_features, hrv_features, weather_features, extra, pv_targets = dataset[i]
    print(hrv_features.shape, weather_features.shape)